<a href="https://colab.research.google.com/github/GianFadiga/MCDA_FS/blob/main/MCDA_UT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# THE MCDA PROJECT: Criação da Ferramenta

## Início

#### Vamos ao código!

In [ ]:
'''
  Aqui toda a execução do código se inicia, não se esqueça de executar
  esta célula que é fundamental para o funcionamento do código
'''

# Importar as bibliotecas a serem utilizadas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
'''
  DICA: Divida os dados qualitativos em categorias (exemplo na CÉLULA 2),
  aqui está uma tabela para auxiliar no processo.
'''

# - Dividir em 5 categorias
# (0 - 20% (Nada desejável),
# 20 - 40% (Pouco desejável),
# 40 - 60% (Médio desejável),
# 60 - 80% (Bem desejável),
# 80 - 100% (Muito desejável))

'\n  DICA: Divida os dados qualitativos em categorias (exemplo na CÉLULA 2),\n  aqui está uma tabela para auxiliar no processo.\n'

# Realizando a montagem dos gráficos

## **CÉLULA 1 - DADOS NUMÉRICOS**

In [ ]:
import pandas as pd
import plotly.express as px

# Leitura do arquivo CSV com a codificação ISO-8859-1
teste_df = pd.read_csv('base_expandida.csv', encoding='ISO-8859-1')

# Extraindo o montante relacional e os tipos de dados
montante_relacional = teste_df.iloc[0].drop('Modelo').astype(float)
tipos_dados = teste_df.iloc[1].drop('Modelo')
teste_df = teste_df.drop([0, 1]).reset_index(drop=True)

# Separando os valores 'BOM' e 'NEUTRO'
valores_bom = teste_df[teste_df['Modelo'] == 'BOM'].iloc[0].drop('Modelo')
valores_neutro = teste_df[teste_df['Modelo'] == 'NEUTRO'].iloc[0].drop('Modelo')

# Convertendo apenas as colunas numéricas para float
for col in tipos_dados.index:
    if tipos_dados[col] == 'number':
        valores_bom[col] = float(valores_bom[col])
        valores_neutro[col] = float(valores_neutro[col])
        teste_df[col] = teste_df[col].astype(float)

# Função para calcular a pontuação baseada na proximidade ao neutro e ao bom
def calcular_pontuacao(valor, bom, neutro, peso):
    if bom - neutro == 0:
        return 0
    elif valor >= neutro:
        return ((valor - neutro) / (bom - neutro)) * peso
    else:
        return -((neutro - valor) / neutro) * peso

# Aplicando a função de pontuação para cada atributo numérico
for col in montante_relacional.index:
    if tipos_dados[col] == 'number':
        teste_df[col + '_pontuacao'] = teste_df.apply(lambda x: calcular_pontuacao(x[col], valores_bom[col], valores_neutro[col], montante_relacional[col]), axis=1)

# Somando as pontuações para obter a pontuação total
teste_df['Pontuacao_Total'] = teste_df[[col + '_pontuacao' for col in montante_relacional.index if tipos_dados[col] == 'number']].sum(axis=1)

# Adicionando classificadores 'BOM' e 'NEUTRO' ao DataFrame
teste_df.loc[teste_df['Modelo'] == 'BOM', 'Color'] = 'Classificadores'
teste_df.loc[teste_df['Modelo'] == 'NEUTRO', 'Color'] = 'Classificadores'
teste_df.loc[~teste_df['Modelo'].isin(['BOM', 'NEUTRO']), 'Color'] = 'Desempenho'

# Ordenando os produtos pela pontuação total
teste_df = teste_df.sort_values(by='Pontuacao_Total', ascending=False).reset_index(drop=True)

# Identificando a pontuação máxima entre os produtos recomendados
pontuacao_maxima = teste_df[teste_df['Color'] == 'Desempenho']['Pontuacao_Total'].max()
produtos_recomendados = teste_df[(teste_df['Color'] == 'Desempenho') & (teste_df['Pontuacao_Total'] == pontuacao_maxima)]

# Exibindo todos os produtos recomendados com detalhes para leigos
for _, produto in produtos_recomendados.iterrows():
    print(f"Produto recomendado: {produto['Modelo']} com pontuação total de {produto['Pontuacao_Total']:.2f}")
    print("Pontos de vantagem e justificativas:")

    pontuacoes_detalhadas = []
    for col in montante_relacional.index:
        if tipos_dados[col] == 'number':
            pontuacao_atributo = produto[col + '_pontuacao']
            if pontuacao_atributo > 0:
                # Contexto adicional para leigos
                justificativa = (
                    f"O produto teve vantagem no atributo '{col}' onde o desempenho neste atributo está {pontuacao_atributo * 100:.0f}% mais próximo do ideal ('BOM') "
                    f"em comparação com o mínimo aceitável ('NEUTRO')."
                )
                pontuacoes_detalhadas.append(justificativa)

    # Exibindo as justificativas detalhadas para o produto
    for detalhe in pontuacoes_detalhadas:
        print(detalhe)
    print("\n")

# Preparando os dados para o gráfico
df_analise = teste_df.copy()

# Definindo cores baseadas nas pontuações
# df_analise['Color'] = df_analise['Pontuacao_Total'].apply(
#     lambda x: 'Classificadores' if x >= 0 else 'Desempenho'
# )

# Definindo as cores com base na pontuação total usando lambda
df_analise['Color'] = df_analise.apply(
    lambda row: 'Classificadores' if row['Modelo'] in ['BOM', 'NEUTRO'] else ('Pontuação Positiva' if row['Pontuacao_Total'] >= 0 else 'Pontuação Negativa'),
    axis=1
)

# Definindo o mapeamento de cores
color_mapping = {
    'Classificadores': 'Classificador',       # Para BOM e NEUTRO
    'Pontuação Positiva': 'Pontuação positiva',  # Para Desempenho
    'Pontuação Negativa': 'Pontuação negativa'   # Para pontuação negativa
}

# Mapeando as cores corretamente
df_analise['Color'] = df_analise['Color'].map(color_mapping)

# Plotando o gráfico
graph = px.bar(df_analise, x='Pontuacao_Total', y='Modelo', orientation='h',
               color='Color', text_auto=False, hover_name='Color',
               hover_data={'Color': False, 'Modelo': False, 'Pontuacao_Total': True},
               color_discrete_sequence=["cornflowerblue", "lightgreen", "palevioletred"])

graph.update_layout(yaxis={'categoryorder': 'total ascending'})
graph.update_xaxes(showticklabels=True)
graph.update_yaxes(showticklabels=True)
graph.update_coloraxes(showscale=False)
graph.update_layout(xaxis_title='')

# Adicionando a linha neutra
graph.add_vline(x=0, line_dash='dash', line_color='black')
graph.add_annotation(
    x=0,
    y=len(df_analise),
    xref="x",
    yref="y",
    text="Linha Neutra<br>(Mínimo Aceitável)",
    showarrow=True,
    font=dict(
        family="Courier New, monospace",
        size=16,
        color="#000000"
    ),
    align="center",
    arrowhead=2,
    arrowsize=1,
    arrowwidth=2,
    arrowcolor="#636363",
    ax=0,
    ay=-45,
    bordercolor="#c7c7c7",
    borderwidth=2,
    borderpad=4,
    bgcolor='white',
    opacity=1
)

graph.update_layout(title='Comparação com pontuação final',
                    legend_title_text='Legenda da relação de cores')
graph.update_layout(hovermode='y unified')

graph.show()


Produto recomendado: Clevo com pontuação total de 5.45
Pontos de vantagem e justificativas:
O produto teve vantagem no atributo 'Memória RAM' onde o desempenho neste atributo está 450% mais próximo do ideal ('BOM') em comparação com o mínimo aceitável ('NEUTRO').
O produto teve vantagem no atributo 'Armazenamento' onde o desempenho neste atributo está 95% mais próximo do ideal ('BOM') em comparação com o mínimo aceitável ('NEUTRO').


Produto recomendado: Eurocom com pontuação total de 5.45
Pontos de vantagem e justificativas:
O produto teve vantagem no atributo 'Memória RAM' onde o desempenho neste atributo está 450% mais próximo do ideal ('BOM') em comparação com o mínimo aceitável ('NEUTRO').
O produto teve vantagem no atributo 'Armazenamento' onde o desempenho neste atributo está 95% mais próximo do ideal ('BOM') em comparação com o mínimo aceitável ('NEUTRO').




In [24]:
import pandas as pd
import plotly.express as px

# Leitura do arquivo CSV com a codificação ISO-8859-1
teste_df = pd.read_csv('base_new.csv', encoding='UTF-8', sep=';')

# Extraindo o montante relacional e os tipos de dados
montante_relacional = teste_df.iloc[0].drop('Modelo').astype(float)
tipos_dados = teste_df.iloc[1].drop('Modelo')
teste_df = teste_df.drop([0, 1]).reset_index(drop=True)

# Separando os valores 'BOM' e 'NEUTRO'
valores_bom = teste_df[teste_df['Modelo'] == 'BOM'].iloc[0].drop('Modelo')
valores_neutro = teste_df[teste_df['Modelo'] == 'NEUTRO'].iloc[0].drop('Modelo')

# Removendo as primeiras três linhas (metadados) e resetando o índice
teste_df = teste_df.drop([0, 1, 2]).reset_index(drop=True)

# Convertendo apenas as colunas numéricas para float
for col in tipos_dados.index:
    if tipos_dados[col] == 'number':
        valores_bom[col] = float(valores_bom[col])
        valores_neutro[col] = float(valores_neutro[col])
        teste_df[col] = teste_df[col].astype(float)

# Função para calcular a pontuação baseada na proximidade ao neutro e ao bom
def calcular_pontuacao(valor, bom, neutro, peso):
    if bom - neutro == 0:
        return 0
    elif valor >= neutro:
        return ((valor - neutro) / (bom - neutro)) * peso
    else:
        return -((neutro - valor) / neutro) * peso

# Aplicando a função de pontuação para cada atributo numérico
for col in montante_relacional.index:
    if tipos_dados[col] == 'number':
        teste_df[col + '_pontuacao'] = teste_df.apply(lambda x: calcular_pontuacao(x[col], valores_bom[col], valores_neutro[col], montante_relacional[col]), axis=1)

# Somando as pontuações para obter a pontuação total
teste_df['Pontuacao_Total'] = teste_df[[col + '_pontuacao' for col in montante_relacional.index if tipos_dados[col] == 'number']].sum(axis=1)

# Adicionando classificadores 'BOM' e 'NEUTRO' ao DataFrame
teste_df.loc[teste_df['Modelo'] == 'BOM', 'Color'] = 'Classificadores'
teste_df.loc[teste_df['Modelo'] == 'NEUTRO', 'Color'] = 'Classificadores'
teste_df.loc[~teste_df['Modelo'].isin(['BOM', 'NEUTRO']), 'Color'] = 'Desempenho'

# Ordenando os produtos pela pontuação total
teste_df = teste_df.sort_values(by='Pontuacao_Total', ascending=False).reset_index(drop=True)

# Identificando a pontuação máxima entre os produtos recomendados
pontuacao_maxima = teste_df[teste_df['Color'] == 'Desempenho']['Pontuacao_Total'].max()
produtos_recomendados = teste_df[(teste_df['Color'] == 'Desempenho') & (teste_df['Pontuacao_Total'] == pontuacao_maxima)]

# Exibindo todos os produtos recomendados com detalhes para leigos
for _, produto in produtos_recomendados.iterrows():
    print(f"Produto recomendado: {produto['Modelo']} com pontuação total de {produto['Pontuacao_Total']:.2f}")
    print("Pontos de vantagem e justificativas:")

    pontuacoes_detalhadas = []
    for col in montante_relacional.index:
        if tipos_dados[col] == 'number':
            pontuacao_atributo = produto[col + '_pontuacao']
            if pontuacao_atributo > 0:
                # Contexto adicional para leigos
                justificativa = (
                    f"O produto teve vantagem no atributo '{col}' onde o desempenho neste atributo está {pontuacao_atributo * 100:.0f}% mais próximo do ideal ('BOM') "
                    f"em comparação com o mínimo aceitável ('NEUTRO')."
                )
                pontuacoes_detalhadas.append(justificativa)

    # Exibindo as justificativas detalhadas para o produto
    for detalhe in pontuacoes_detalhadas:
        print(detalhe)
    print("\n")

# Preparando os dados para o gráfico
df_analise = teste_df.copy()

# Definindo cores baseadas nas pontuações
df_analise['Color'] = df_analise.apply(
    lambda row: 'Classificadores' if row['Modelo'] in ['BOM', 'NEUTRO'] else ('Pontuação Positiva' if row['Pontuacao_Total'] >= 0 else 'Pontuação Negativa'),
    axis=1
)

# Definindo o mapeamento de cores
color_mapping = {
    'Classificadores': 'Classificador',       # Para BOM e NEUTRO
    'Pontuação Positiva': 'Pontuação positiva',  # Para Desempenho
    'Pontuação Negativa': 'Pontuação negativa'   # Para pontuação negativa
}

# Mapeando as cores corretamente
df_analise['Color'] = df_analise['Color'].map(color_mapping)

# Plotando o gráfico
graph = px.bar(df_analise, x='Pontuacao_Total', y='Modelo', orientation='h',
               color='Color', text_auto=False, hover_name='Color',
               hover_data={'Color': False, 'Modelo': False, 'Pontuacao_Total': True},
               color_discrete_sequence=["cornflowerblue", "lightgreen", "palevioletred"])

graph.update_layout(yaxis={'categoryorder': 'total ascending'})
graph.update_xaxes(showticklabels=True)
graph.update_yaxes(showticklabels=True)
graph.update_coloraxes(showscale=False)
graph.update_layout(xaxis_title='')

# Adicionando a linha neutra
graph.add_vline(x=0, line_dash='dash', line_color='black')
graph.add_annotation(
    x=0,
    y=len(df_analise),
    xref="x",
    yref="y",
    text="Linha Neutra<br>(Mínimo Aceitável)",
    showarrow=True,
    font=dict(
        family="Courier New, monospace",
        size=16,
        color="#000000"
    ),
    align="center",
    arrowhead=2,
    arrowsize=1,
    arrowwidth=2,
    arrowcolor="#636363",
    ax=0,
    ay=-45,
    bordercolor="#c7c7c7",
    borderwidth=2,
    borderpad=4,
    bgcolor='white',
    opacity=1
)

graph.update_layout(title='Comparação com pontuação final',
                    legend_title_text='Legenda da relação de cores')
graph.update_layout(hovermode='y unified')

graph.show()

Produto recomendado: Apple com pontuação total de 1.41
Pontos de vantagem e justificativas:
O produto teve vantagem no atributo 'Memória RAM' onde o desempenho neste atributo está 90% mais próximo do ideal ('BOM') em comparação com o mínimo aceitável ('NEUTRO').
O produto teve vantagem no atributo 'Armazenamento' onde o desempenho neste atributo está 21% mais próximo do ideal ('BOM') em comparação com o mínimo aceitável ('NEUTRO').
O produto teve vantagem no atributo 'Bateria mAh' onde o desempenho neste atributo está 30% mais próximo do ideal ('BOM') em comparação com o mínimo aceitável ('NEUTRO').




In [25]:
teste_df.head(10)

,Modelo,Memória RAM,Armazenamento,Tamanho de tela,Bateria mAh,Cor,Bluetooth,Memória RAM_pontuacao,Armazenamento_pontuacao,Tamanho de tela_pontuacao,Bateria mAh_pontuacao,Pontuacao_Total,Color
0,Apple,16.0,1000.0,16.0,6000.0,Cinza,TRUE,0.90,0.209524,0,0.30,1.409524,Desempenho
1,Positivo,16.0,120.0,16.0,5000.0,Verde,FALSE,0.90,0.000000,0,0.20,1.100000,Desempenho
2,Asus,12.0,512.0,13.0,4500.0,Preto,TRUE,0.60,0.093333,0,0.15,0.843333,Desempenho
3,Samsung,8.0,240.0,14.0,4000.0,Azul,TRUE,0.30,0.028571,0,0.10,0.428571,Desempenho
4,HP,8.0,256.0,14.0,3500.0,Prata,TRUE,0.30,0.032381,0,0.05,0.382381,Desempenho
5,DELL,6.0,480.0,15.0,3000.0,Vermelho,FALSE,0.15,0.085714,0,0.00,0.235714,Desempenho
6,Acer,2.0,960.0,14.0,4000.0,Preto,TRUE,-0.15,0.200000,0,0.10,0.150000,Desempenho
7,Lenovo,4.0,480.0,15.0,3000.0,Prata,TRUE,0.00,0.085714,0,0.00,0.085714,Desempenho


In [26]:
import pandas as pd
import plotly.express as px

# Leitura e preparação dos dados
teste_df = pd.read_csv('base_new.csv', encoding='UTF-8', sep=';')

# Extração de metadados
weights = teste_df.iloc[0].drop('Modelo').astype(float)      # Linha 0: pesos
data_types = teste_df.iloc[1].drop('Modelo')                 # Linha 1: tipos de dados
proportionality = teste_df.iloc[2].drop('Modelo')            # Linha 2: tipo de proporcionalidade

# Remover linhas de metadados e resetar índice
teste_df = teste_df.iloc[3:].reset_index(drop=True)

# Extrair valores de referência
bom_values = teste_df[teste_df['Modelo'] == 'BOM'].iloc[0].drop('Modelo')
neutro_values = teste_df[teste_df['Modelo'] == 'NEUTRO'].iloc[0].drop('Modelo')

# Remover linhas de classificação do DataFrame principal
teste_df = teste_df[~teste_df['Modelo'].isin(['BOM', 'NEUTRO'])].reset_index(drop=True)

# Converter colunas numéricas
numeric_cols = data_types[data_types == 'number'].index
for col in numeric_cols:
    bom_values[col] = float(bom_values[col])
    neutro_values[col] = float(neutro_values[col])
    teste_df[col] = teste_df[col].astype(float)

# Função de cálculo de pontuação aprimorada
def calculate_score(row, col):
    valor = row[col]
    bom = bom_values[col]
    neutro = neutro_values[col]
    peso = weights[col]
    prop_type = proportionality[col]

    # Caso valores iguais
    if bom == neutro:
        return 0

    # Determinar direção da proporcionalidade
    if prop_type == 'i_proportional':
        bom, neutro = neutro, bom  # Inverte os valores para cálculo

    # Cálculo da pontuação normalizada
    if valor >= bom:
        return peso  # Pontuação máxima
    elif valor <= neutro:
        return -peso  # Penalização máxima
    else:
        # Interpolação linear
        return ((valor - neutro) / (bom - neutro)) * peso

# Calcular pontuações para cada atributo
for col in numeric_cols:
    teste_df[f'{col}_score'] = teste_df.apply(lambda x: calculate_score(x, col), axis=1)

# Calcular pontuação total
score_columns = [f'{col}_score' for col in numeric_cols]
teste_df['Total_Score'] = teste_df[score_columns].sum(axis=1)

# Classificação e ordenação
teste_df = teste_df.sort_values('Total_Score', ascending=False).reset_index(drop=True)

# Identificar melhor produto
best_score = teste_df['Total_Score'].max()
best_products = teste_df[teste_df['Total_Score'] == best_score]

# Função para gerar justificativas detalhadas
def generate_justification(product):
    justifications = []
    for col in numeric_cols:
        score = product[f'{col}_score']
        prop_type = proportionality[col]

        if score > 0:
            direction = "maior" if prop_type == 'proportional' else "menor"
            comparison = "superou" if prop_type == 'proportional' else "ficou abaixo"
            justifications.append(
                f"{col}: Valor {product[col]} ({comparison} o NEUTRO ({neutro_values[col]}), "
                f"ideal é {direction} que {bom_values[col]} (Contribuição: +{score:.2f})"
            )
    return justifications

# Exibir resultados
print("🏆 Melhores produtos encontrados:")
for _, product in best_products.iterrows():
    print(f"\n📌 {product['Modelo']} (Pontuação: {product['Total_Score']:.2f})")
    print("📈 Pontos fortes:")
    for justification in generate_justification(product):
        print(f"  ✓ {justification}")

# Visualização gráfica aprimorada
df_viz = teste_df.copy()
df_viz['Tipo'] = df_viz.apply(
    lambda x: 'Referência' if x['Modelo'] in ['BOM', 'NEUTRO'] else 'Produto',
    axis=1
)

fig = px.bar(df_viz,
             x='Total_Score',
             y='Modelo',
             color='Tipo',
             color_discrete_map={'Produto': '#1f77b4', 'Referência': '#ff7f0e'},
             text='Total_Score',
             orientation='h')

fig.update_layout(
    title='Análise Comparativa de Produtos',
    xaxis_title='Pontuação Total',
    yaxis_title='Modelo',
    template='plotly_white',
    hovermode='y'
)

fig.add_vline(x=0, line_dash='dot', line_color='grey')
fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig.show()

🏆 Melhores produtos encontrados:

📌 Apple (Pontuação: 0.70)
📈 Pontos fortes:
  ✓ Memória RAM: Valor 16.0 (superou o NEUTRO (4.0), ideal é maior que 8.0 (Contribuição: +0.30)
  ✓ Armazenamento: Valor 1000.0 (superou o NEUTRO (120.0), ideal é maior que 960.0 (Contribuição: +0.20)
  ✓ Bateria mAh: Valor 6000.0 (superou o NEUTRO (3000.0), ideal é maior que 5000.0 (Contribuição: +0.20)


In [18]:
teste_df.head(10)

,Modelo,Memória RAM,Armazenamento,Tamanho de tela,Bateria mAh,Cor,Bluetooth,Memória RAM_score,Armazenamento_score,Tamanho de tela_score,Bateria mAh_score,Total_Score
0,Apple,16.0,1000.0,16.0,6000.0,Cinza,TRUE,0.30,0.200000,0,0.20,7.000000e-01
1,Asus,12.0,512.0,13.0,4500.0,Preto,TRUE,0.30,0.093333,0,0.15,5.433333e-01
2,Samsung,8.0,240.0,14.0,4000.0,Azul,TRUE,0.30,0.028571,0,0.10,4.285714e-01
3,HP,8.0,256.0,14.0,3500.0,Prata,TRUE,0.30,0.032381,0,0.05,3.823810e-01
4,Positivo,16.0,120.0,16.0,5000.0,Verde,FALSE,0.30,-0.200000,0,0.20,3.000000e-01
5,DELL,6.0,480.0,15.0,3000.0,Vermelho,FALSE,0.15,0.085714,0,-0.20,3.571429e-02
6,Acer,2.0,960.0,14.0,4000.0,Preto,TRUE,-0.30,0.200000,0,0.10,2.775558e-17
7,Lenovo,4.0,480.0,15.0,3000.0,Prata,TRUE,-0.30,0.085714,0,-0.20,-4.142857e-01


In [20]:
import pandas as pd
import plotly.express as px

# Leitura do arquivo CSV
teste_df = pd.read_csv('base_new.csv', encoding='UTF-8', sep=';')

# Extração dos metadados (3 primeiras linhas)
montante_relacional = teste_df.iloc[0].drop('Modelo').astype(float)  # Linha 0: pesos
tipos_dados = teste_df.iloc[1].drop('Modelo')                       # Linha 1: tipos de dados
proporcionalidade = teste_df.iloc[2].drop('Modelo')                 # Linha 2: tipo de proporcionalidade

# Remover TODAS as linhas de metadados (linhas 0, 1 e 2)
teste_df = teste_df.drop([0, 1, 2]).reset_index(drop=True)

# Separar valores BOM e NEUTRO (agora nas primeiras linhas após remover metadados)
valores_bom = teste_df[teste_df['Modelo'] == 'BOM'].iloc[0].drop('Modelo')
valores_neutro = teste_df[teste_df['Modelo'] == 'NEUTRO'].iloc[0].drop('Modelo')

# Remover BOM/NEUTRO do DataFrame principal para processamento
teste_df = teste_df[~teste_df['Modelo'].isin(['BOM', 'NEUTRO'])].reset_index(drop=True)

# Converter apenas colunas numéricas (baseado na linha de tipos_dados)
for col in tipos_dados.index:
    if tipos_dados[col] == 'number':
        # Converter valores BOM/NEUTRO primeiro
        valores_bom[col] = float(valores_bom[col])
        valores_neutro[col] = float(valores_neutro[col])
        # Converter coluna no DataFrame principal
        teste_df[col] = teste_df[col].astype(float)

# ... (restante do código de cálculo de pontuações e gráfico permanece igual) ...

# Adicionar BOM/NEUTRO de volta ao DataFrame para visualização
teste_df = pd.concat([
    teste_df,
    pd.DataFrame([{'Modelo': 'BOM', **valores_bom}]),
    pd.DataFrame([{'Modelo': 'NEUTRO', **valores_neutro}])
], ignore_index=True)

# Configurar cores
teste_df['Color'] = teste_df.apply(
    lambda row: 'Classificador' if row['Modelo'] in ['BOM', 'NEUTRO']
    else 'Pontuação positiva' if row['Pontuacao_Total'] >= 0
    else 'Pontuação negativa', axis=1
)

# Gerar gráfico (mantendo configuração original)
graph = px.bar(teste_df, x='Pontuacao_Total', y='Modelo', color='Color',
               color_discrete_map={
                   'Classificador': 'cornflowerblue',
                   'Pontuação positiva': 'lightgreen',
                   'Pontuação negativa': 'palevioletred'
               },
               orientation='h')

graph.add_vline(x=0, line_dash='dash', line_color='black')
graph.update_layout(yaxis={'categoryorder': 'total ascending'})
graph.show()

KeyError: 'Pontuacao_Total'

## **CÉLULA 2 - DADOS NÃO NUMÉRICOS**

In [ ]:
df = pd.DataFrame()

# Situação/objeto exemplo: compra de um celular
# Começar considerando cores como categoria a ser analisada
# Considerar Vermelho como Muito desejável, Preto como Neutro e Verde como Nada desejável

# A ideia é a criação de diversos métodos de análise e com gráficos a serem analisados
# No caso, cada cor receberia um slider para que o usuário selecione a sua importância
# Considerando valores entre os intervalos dados (trabalhando na base de 0 até 100, logo, números decimais)
# Onde 0 é o mínimo desejável e 1 é o máximo desejável

'''
  Instruções de uso:
    - Com o dataframe já carregado da CÉLULA 1, basta alterar os dados de teste
    (placeholder) aqui presentes e executar o código.
    DICA: Crie cópias da célula, assim execute o código uma vez para cada dado
    qualitativo que desejar comparar.
'''

# Altere aqui! - Início
df['Cores'] = ['Vermelho', 'Verde', 'Branco', 'Preto', 'BOM', 'NEUTRO']
df['Valor Cores'] = [90, 10, 40, 50, 80, 50]
# Altere aqui! 0 Final

df['Tam_scatter'] = [1]*len(df)
symbols = ['diamond-wide']*len(df)

df['Valor X'] = [0]*len(df)

val_neutro = df['Valor Cores'][df.index[df['Cores'] == 'NEUTRO'][0]]
val_bom = df['Valor Cores'][df.index[df['Cores'] == 'BOM'][0]]

df['Definicao'] = pd.Series(dtype='str')
for i in range(len(df)):
    if df['Cores'][i] == 'NEUTRO' or df['Cores'][i] == 'BOM':
        df.loc[i, 'Definicao'] = 'Classificador'
    else:
        df['Definicao'] = 'Desempenho'
# Diversas cores para daltônicos

cores1 = ['red', 'blue']
cores2 = ['pink', 'green']
cores3 = ['orange', 'purple']
cores4 = ['orange', 'gray']

graph = px.scatter(df, x='Valor X', y='Valor Cores', title='Comparação com Indicadores de Desempenho e Classificadores',
                   color='Cores', symbol_sequence=symbols,
                   hover_data={'Cores':False, 'Valor Cores':False, 'Valor X':False,
                               'Valor Referencial': (df['Cores'])})

graph.update_traces(marker={'size': 20})
graph.update_xaxes(fixedrange=True, title=None)
graph.update_yaxes(fixedrange=True, title=None)

for i in range(len(df)):
    if df['Definicao'][i] == 'Classificador' and df['Cores'][i] == 'NEUTRO':
        graph.add_hline(y=df['Valor Cores'][i], line_color=cores3[1], line_width=3, x0=0.3, x1=0.7)
    elif df['Definicao'][i] == 'Classificador' and df['Cores'][i] == 'BOM':
        graph.add_hline(y=df['Valor Cores'][i], line_color=cores3[0], line_width=3, x0=0.3, x1=0.7)


graph.update_xaxes(showticklabels=False, showgrid=False)
graph.update_yaxes(showticklabels=False, showgrid=False)

graph.update_layout(legend=dict(
    orientation='h',
    yanchor='bottom',
    y=-0.2,
    xanchor='right',
    x=1,
    title_text='Variáveis e Cores'
))

graph.add_vline(x=0, y0=0, y1=1)
graph.add_annotation(
    x=-0.2,
    y=110,
    xref='x',
    yref='y',
    text=f'Linha Neutra<br>(Mínimo Aceitável)',
    showarrow=False,
    font=dict(
            family="Courier New, monospace",
            size=16,
            color="#ffffff"
            ),
        align="center",
        bordercolor="#c7c7c7",
        borderwidth=2,
        borderpad=4,
        bgcolor=cores3[1],
        opacity=1
)
graph.add_annotation(
    x=0.2,
    y=110,
    xref='x',
    yref='y',
    text=f'Linha Bom<br>(Muito Desejável)',
    showarrow=False,
    font=dict(
            family="Courier New, monospace",
            size=16,
            color="#ffffff"
            ),
        align="center",
        bordercolor="#c7c7c7",
        borderwidth=2,
        borderpad=4,
        bgcolor=cores3[0],
        opacity=1
)
graph.show()